In [ ]:
import pandas as pd
import re

from utils.column_merge_utils import column_merge

In [ ]:
column_merge_rules = [
    [['단지코드', 'k-아파트코드'], ['단지코드']],
    [['시도', '주소(시도)k-apt주소split'], ['시도']],
    [['시군구', '주소(시군구)'], ['시군구']],
    [['동리', '주소(읍면동)'], ['동리']],
    [['단지명', 'k-아파트명'], ['단지명']],
    [['단지분류', 'k-단지분류(아파트,주상복합등등)'], ['단지분류', '보조_단지분류']],
    [['동수', 'k-전체동수'], ['동수']],
    [['세대수', 'k-전체세대수'], ['세대수', '보조_세대수']],
    [['관리방식', 'k-관리방식'], ['관리방식', '보조_관리방식']],
    [['난방방식', 'k-난방방식'], ['난방방식', '보조_난방방식']],
    [['복도유형', 'k-복도유형'], ['복도유형', '보조_복도유형']],
    [['시공사', 'k-건설사(시공사)'], ['시공사']],
    [['시행사', 'k-시행사'], ['시행사']],
]

# 파일 읽기

In [ ]:
kapt_excel_path = "../input/20251205_단지_기본정보.xlsx"
kapt_csv_path = "../input/kapt.csv"

sapt_original_path = "../input/서울시 공동주택 아파트 정보.csv"
sapt_csv_path = "../input/sapt.csv"

In [ ]:
kapt = pd.read_excel(kapt_excel_path)
kapt.to_csv(kapt_csv_path, index=False)

sapt = pd.read_csv(sapt_original_path, encoding='EUC-KR')
sapt.to_csv(sapt_csv_path, index=False)

In [ ]:
kapt = pd.read_csv(kapt_csv_path, dtype='string')
kapt = kapt[kapt['시도']=="서울특별시"] # 서울시 데이터만 이용
print(f"kapt : {len(kapt)}")

sapt = pd.read_csv(sapt_csv_path, dtype='string')
print(f"sapt : {len(sapt)}")

In [ ]:
# 원본데이터에 
dirty_columns = [c for c in sapt.columns if "Unnamed" in c]
print(f"number of columns to drop in sapt : {len(dirty_columns)}")
sapt = sapt.drop(columns=dirty_columns)

# 병합
kapt['단지코드']와 sapt['k-아파트코드']를 기준 키로 두 테이블을 full outer join

In [ ]:
merged = kapt.merge(    # outer join
    sapt,
    left_on='단지코드',
    right_on='k-아파트코드',
    how='outer',
    indicator=True
)
merged['병합종류'] = merged['_merge'].map({
    'both': '성공',
    'left_only': '2번',
    'right_only': '3번'
})
col_to_move = merged.pop('병합종류')
merged.insert(0,'병합종류',col_to_move)

matched = (             # 매칭에 성공한 것
    merged.loc[merged['_merge'] == 'both']
    .drop(columns = ['_merge'])
    .copy()
)

unmatched_kapt = (      # 매칭에 실패한 kapt
    merged.loc[merged['_merge'] == 'left_only']
    .drop(columns = ['_merge'])
    .copy()
)

unmatched_sapt = (      # 매칭에 실패한 sapt
    merged.loc[merged['_merge'] == 'right_only']
    .drop(columns = ['_merge'])
    .copy()
)

merged=merged.drop(columns=('_merge'))

print(f"merged : {len(merged)}")
print(f"matched : {len(matched)}")
print(f"unmatched_kapt : {len(unmatched_kapt)}")
print(f"unmatched_sapt : {len(unmatched_sapt)}")

# 컬럼 병합
kapt 테이블과 sapt 테이블이 공통으로 가지는 컬럼이 많으므로, 둘을 병합한 테이블에서 중복컬럼(이름만 다르고 내용은 같은 컬럼)이 발생합.<br>
중복컬럼이 있을 때, 두 컬럼값이 같으면 하나의 컬럼으로 병합.<br>
병합이 불가능한 경우에는 kapt의 값을 병합된 컬럼의 값으로 하고, 옆에 보조컬럼을 생성해 sapt의 컬럼병합 실패한 값을 기록

In [ ]:
merged.loc[merged['주소(시도)k-apt주소split'].eq('서울'),'주소(시도)k-apt주소split']="서울특별시" # sapt에서 ['시도']가 '서울', '서울특별시'인 경우가 있어 '서울특별시'로 통일

merged = column_merge(merged, column_merge_rules)

#### 도로명주소 컬럼 병합
kapt와 sapt의 도로명주소 컬럼 병합<br>
한쪽이 NaN이거나 공백일 경우 그렇지 않은 값을 이용해 병합<br>
두 값이 같을 경우 병합<br>
kapt주소가 sapt주소를 포함하는 경우, kapt주소를 이용해 병합<br>
예시) kapt_도로명주소 = "강서로 1-1, 강서로 1-2", sapt_도로명주소 = "강서로 1-1"

In [ ]:
def merge_도로명주소(kapt_도로명주소, sapt_도로명주소):
    merged_도로명주소 = ''
    보조_도로명주소 = ''
    
    if(pd.isna(kapt_도로명주소) or kapt_도로명주소==''):    
        merged_도로명주소=sapt_도로명주소
    elif(pd.isna(sapt_도로명주소) or sapt_도로명주소==''):    
        merged_도로명주소=kapt_도로명주소
    elif(kapt_도로명주소==sapt_도로명주소):   
        merged_도로명주소=kapt_도로명주소
    elif(sapt_도로명주소 in kapt_도로명주소):    # kapt주소가 sapt주소를 포함하는 경우
        merged_도로명주소=kapt_도로명주소
    else:
        merged_도로명주소 = kapt_도로명주소
        보조_도로명주소 = sapt_도로명주소
    return merged_도로명주소, 보조_도로명주소

merged = merged.rename(columns={
    '도로명주소' : 'kapt_도로명주소',
    'kapt도로명주소' : 'sapt_도로명주소'
})

merged[['도로명주소', '보조_도로명주소']] = merged.apply(
    lambda row: merge_도로명주소(row['kapt_도로명주소'], row['sapt_도로명주소']),
    axis=1,
    result_type='expand'
)

col = merged.pop('도로명주소')
merged.insert(merged.columns.get_loc('kapt_도로명주소')+1,'도로명주소',col)
col = merged.pop('보조_도로명주소')
merged.insert(merged.columns.get_loc('kapt_도로명주소')+2,'보조_도로명주소',col)

merged=merged.drop(columns=['kapt_도로명주소', 'sapt_도로명주소'])

#### 분양형태 컬럼 병합

In [ ]:
def merge_분양형태(kapt_분양형태, sapt_분양형태):
    merged_분양형태='' 
    보조_분양형태=''     
    
    if(pd.isna(kapt_분양형태)): merged_분양형태 = sapt_분양형태
    elif(pd.isna(sapt_분양형태)): merged_분양형태 = kapt_분양형태
    elif(
        (kapt_분양형태==sapt_분양형태)
        or (kapt_분양형태=='혼합' and sapt_분양형태=='기타')    # kapt에서의 "혼합"은 sapt에서의 "기타"와 같은 의미임
        ):
        merged_분양형태=kapt_분양형태
    else:
        merged_분양형태=kapt_분양형태
        보조_분양형태=sapt_분양형태
    return merged_분양형태, 보조_분양형태

merged = merged.rename(columns ={
    '분양형태' : 'kapt_분양형태',
    'k-세대타입(분양형태)' : 'sapt_분양형태'
    }) 

merged[['분양형태','보조_분양형태']] = merged.apply( 
    lambda row: merge_분양형태(row['kapt_분양형태'], row['sapt_분양형태']),
    axis=1,
    result_type='expand'
)

col = merged.pop('분양형태')
merged.insert(merged.columns.get_loc('kapt_분양형태')+1,'분양형태',col)
col = merged.pop('보조_분양형태')
merged.insert(merged.columns.get_loc('kapt_분양형태')+2,'보조_분양형태',col)

merged=merged.drop(columns=['kapt_분양형태', 'sapt_분양형태'])

#### 경비관리-관리방식 컬럼 병합

In [ ]:
def merge_경비관리방식(kapt_경비관리방식, sapt_경비관리방식):
    merged_경비관리방식=''
    보조_경비관리방식=''
    
    if(pd.isna(kapt_경비관리방식) or kapt_경비관리방식==''): 
        merged_경비관리방식=sapt_경비관리방식
    elif(pd.isna(sapt_경비관리방식) or sapt_경비관리방식==''): 
        merged_경비관리방식=kapt_경비관리방식
    elif(
        (kapt_경비관리방식=="위탁관리" and sapt_경비관리방식=="위탁")
        or (kapt_경비관리방식=="자치관리" and sapt_경비관리방식=="직영")
        ):
        merged_경비관리방식 = kapt_경비관리방식
    else:
        merged_경비관리방식=kapt_경비관리방식
        보조_경비관리방식=sapt_경비관리방식
    return merged_경비관리방식, 보조_경비관리방식

merged = merged.rename(columns={
    '경비관리-관리방식' : 'kapt_경비관리방식',
    '경비비관리형태' : 'sapt_경비관리방식'
})

merged[['경비관리방식', '보조_경비관리방식']] = merged.apply(
    lambda row: merge_경비관리방식(row['kapt_경비관리방식'], row['sapt_경비관리방식']),
    axis=1,
    result_type='expand'
)

col = merged.pop('경비관리방식')
merged.insert(merged.columns.get_loc('kapt_경비관리방식')+1,'경비관리방식',col)
col = merged.pop('보조_경비관리방식')
merged.insert(merged.columns.get_loc('kapt_경비관리방식')+2,'보조_경비관리방식',col)

merged=merged.drop(columns=['kapt_경비관리방식', 'sapt_경비관리방식'])

#### 청소관리-관리방식 컬럼 병합

In [ ]:
def merge_청소관리방식(kapt_청소관리방식, sapt_청소관리방식):
    merged_청소관리방식=''
    보조_청소관리방식=''
    
    if(pd.isna(kapt_청소관리방식) or kapt_청소관리방식==''): 
        merged_청소관리방식=sapt_청소관리방식
    elif(pd.isna(sapt_청소관리방식) or sapt_청소관리방식==''): 
        merged_청소관리방식=kapt_청소관리방식
    elif(
        (kapt_청소관리방식=="위탁관리" and sapt_청소관리방식=="위탁")
        or (kapt_청소관리방식=="자치관리" and sapt_청소관리방식=="직영")
        ):
        merged_청소관리방식 = kapt_청소관리방식
    else:
        merged_청소관리방식=kapt_청소관리방식
        보조_청소관리방식=sapt_청소관리방식
    return merged_청소관리방식, 보조_청소관리방식

merged = merged.rename(columns={
    '청소관리-관리방식' : 'kapt_청소관리방식',
    '청소비관리형태' : 'sapt_청소관리방식'
})

merged[['청소관리방식','보조_청소관리방식']] = merged.apply(
    lambda row: merge_청소관리방식(row['kapt_청소관리방식'], row['sapt_청소관리방식']),
    axis=1,
    result_type='expand'
)

col = merged.pop('청소관리방식')
merged.insert(merged.columns.get_loc('kapt_청소관리방식')+1,'청소관리방식',col)
col = merged.pop('보조_청소관리방식')
merged.insert(merged.columns.get_loc('kapt_청소관리방식')+2,'보조_청소관리방식',col)

merged=merged.drop(columns=['kapt_청소관리방식', 'sapt_청소관리방식'])

# 디자인 추가

In [ ]:
def apt_parse_법정동주소(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)    
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)     # 주소값이 없을 경우에 널값 리턴
    
    시도 = pd.NA
    군구 = pd.NA
    동리= pd.NA
    번지= pd.NA
    상세= pd.NA
    중복주소= pd.NA
    addr_rest = addr
    
    # 주소 파싱
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+)\s*(.*)", addr_rest)
    if match_groups:
        동리 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)(?:번지)?-?\s*(.*)", addr_rest)
    if match_groups:
        번지 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    if addr_rest:
        if addr_rest.startswith(','):
            중복주소 = addr_rest.lstrip(',').strip()
        else:
            상세 = addr_rest
        
    return (시도, 군구, 동리, 번지, 상세, 중복주소)

merged[
    ['법정동주소(시도)', '법정동주소(군구)', '법정동주소(동리)','법정동주소(번지)', '법정동주소(상세)', '법정동주소(중복주소)']
] = merged['법정동주소'].apply(
    lambda x: pd.Series(apt_parse_법정동주소(x))
)

valid_mask = (
    merged['법정동주소'].notna()
    & merged['법정동주소'].apply(lambda val : str(val).strip()!='')
)
법정동주소_dup_mask = (
    valid_mask
    & merged.duplicated(subset=['법정동주소(시도)','법정동주소(군구)','법정동주소(동리)','법정동주소(번지)'], keep=False)
)

def highlight_dup_법정동주소(row):
    styles = [''] * len(row)
    color = "#fc3b3b"
    
    if not 법정동주소_dup_mask.loc[row.name]:
        return styles
    
    for i, col in enumerate(row.index):
        if col=="법정동주소":
            styles[i] = f'background-color: {color}'
            
    return styles

merged = merged.drop(columns = ['법정동주소(시도)', '법정동주소(군구)', '법정동주소(동리)','법정동주소(번지)', '법정동주소(상세)', '법정동주소(중복주소)'])

In [ ]:
def apt_parse_도로명주소(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    도로명 = pd.NA
    건물번호 = pd.NA
    중복주소 = pd.NA
    상세 = pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+(?:로|길))\s*(.*)", addr_rest)
    if match_groups:
        도로명 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)\s*(.*)", addr_rest)
    if match_groups:
        건물번호 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    if addr_rest:
        if addr_rest.startswith(','):
            중복주소 = addr_rest.lstrip(',').strip()
        else:
            상세 = addr_rest
        
    return (시도, 군구, 도로명, 건물번호, 상세, 중복주소)

merged[
    ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)', '도로명주소(상세)', '도로명주소(중복주소)']
] = merged['도로명주소'].apply(
    lambda x: pd.Series(apt_parse_도로명주소(x))
)

valid_mask = (
    merged['도로명주소'].notna()
    & merged['도로명주소'].apply(lambda val : str(val).strip()!='')
)
도로명주소_dup_mask = (
    valid_mask
    & merged.duplicated(subset=['도로명주소(시도)','도로명주소(군구)','도로명주소(도로명)','도로명주소(건물번호)'], keep=False)
)
print(도로명주소_dup_mask.sum())

def highlight_dup_도로명주소(row):
    styles = [''] * len(row)
    color = "#fc3b3b"
    
    if not 도로명주소_dup_mask.loc[row.name]:
        return styles
    
    for i, col in enumerate(row.index):
        if col=="도로명주소":
            styles[i] = f'background-color: {color}'
            
    return styles

merged = merged.drop(columns=['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)', '도로명주소(상세)', '도로명주소(중복주소)'])

In [ ]:
def highlight_merge_type(row):
    styles = ['']*len(row)
    
    if row['병합종류']=='2번':
        color = "#dff3f6"
    elif row['병합종류']=='3번':
        color = "#ebe8fa"
    else:
        return styles

    for i, col in enumerate(row.index):
        styles[i] = f'background-color: {color}'
        
    return styles

In [ ]:
def highlight_merge_fail(row, main_col, sub_col, color="#ffd900"):
    styles = [''] * len(row)

    if pd.notna(row[sub_col]) and row[sub_col] != '':
        for i, col in enumerate(row.index):
            if col in [main_col, sub_col]:
                styles[i] = f'background-color: {color}'

    return styles

In [ ]:
styled = merged.style

styled = styled.apply(
    highlight_merge_type,
    axis=1
)

# styled = styled.apply(
#     highlight_merge_fail,
#     axis=1,
#     main_col='시도',
#     sub_col='보조_시도'
# )

# styled = styled.apply(
#     highlight_merge_fail,
#     axis=1,
#     main_col='시군구',
#     sub_col='보조_시군구'
# )

# styled = styled.apply(
#     highlight_merge_fail,
#     axis=1,
#     main_col='동리',
#     sub_col='보조_동리'
# )

# styled = styled.apply(
#     highlight_merge_fail,
#     axis=1,
#     main_col='단지명',
#     sub_col='보조_단지명'
# )

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='단지분류',
    sub_col='보조_단지분류'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='도로명주소',
    sub_col='보조_도로명주소'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='분양형태',
    sub_col='보조_분양형태'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='세대수',
    sub_col='보조_세대수'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='관리방식',
    sub_col='보조_관리방식'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='난방방식',
    sub_col='보조_난방방식'
)

# styled = styled.apply(
#     highlight_merge_fail,
#     axis=1,
#     main_col='시행사',
#     sub_col='보조_시행사'
# )

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='경비관리방식',
    sub_col='보조_경비관리방식'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='청소관리방식',
    sub_col='보조_청소관리방식'
)

styled = styled.apply(
    highlight_dup_도로명주소,
    axis=1
)

styled = styled.apply(
    highlight_dup_법정동주소,
    axis=1
)

# 파일 저장

In [ ]:
styled.to_excel("../output/kapt_sapt_병합됨.xlsx", index = False)
unmatched_kapt.to_excel("../output/kapt_병합실패.xlsx", index = False)
unmatched_sapt.to_excel("../output/sapt_병합실패.xlsx", index = False)